# Neural Networks with PyTorch

The materials in this notebook are sourced from [PyTorch's 60 Minute Blitz](https://pytorch.org/tutorials/beginner/blitz/neural_networks_tutorial.html#sphx-glr-beginner-blitz-neural-networks-tutorial-py) with annotations from their [docs](https://pytorch.org/docs/stable/).

We will construct a network that classifies digit images.

<img src="https://pytorch.org/tutorials/_images/mnist.png" />

It is a simple feed-forward network. It takes the input, feeds it through several layers one after the other, and then finally gives the output.

A typical training procedure for a neural network is as follows:
* Define the neural network that has some learnable parameters (or weights)
* Iterate over a dataset of inputs
* Process input through the network
* Compute the loss (how far is the output from being correct)
* Propagate gradients back into the network’s parameters
* Update the weights of the network, typically using a simple update rule: weight = weight - learning_rate * gradient


## Defining the Network

In [2]:
import torch
import torch.nn as nn
import torch.nn.functional as F

class Net(nn.Module):
    
    def __init__(self):
        """
        First convolution layer
          1 input image channel
          6 output channels
          5x5 square convolution kernel
        Second convolution layer
          6 input image channel
          16 output channels
          5x5 square convolution kernel
          
        First fully connected layer
          1 input image channel
          6 output channels
          5x5 square convolution kernel
        Second fully connected layer
          1 input image channel
          6 output channels
          5x5 square convolution kernel
        Third fully connected layer
          1 input image channel
          6 output channels
          5x5 square convolution kernel
        """ 
        super(Net, self).__init__()
        
        self.conv1 = nn.Conv2d(1, 6, 5)
        self.conv2 = nn.Conv2d(6, 16, 5)
        
        self.fc1 = nn.Linear(16 * 5 * 5, 120)
        self.fc2 = nn.Linear(120, 84)
        self.fc3 = nn.Linear(84, 10)
        
        
    def forward(self, x):
        # Max pooling over a (2, 2) window
        x = F.max_pool2d(F.relu(self.conv1(x)), (2, 2))
        # If the size is a square you can only specify a single number
        x = F.max_pool2d(F.relu(self.conv2(x)), 2)
        x = x.view(-1, self.num_flat_features(x))
        x = F.relu(self.fc1(x))
        x = F.relu(self.fc2(x))
        x = self.fc3(x)
        return x

    def num_flat_features(self, x):
        size = x.size()[1:]  # all dimensions except the batch dimension
        num_features = 1
        for s in size:
            num_features *= s
        return num_features


net = Net()
print(net)

ModuleNotFoundError: No module named 'torch'

### torch.nn.Conv2d

Applies a 2D **convolution** over an input signal composed of several input planes.

>Slides an nxn kernel across the input matrix performing mathematical convolution on the submatrix that the kernel covers. The values or weights of the kernel matrix are learned by backpropogation.

In the simplest case, the output value of the layer with input size $(N, C_{in}, H, W)$ and output $(N, C_{out}, H_{out}, W_{out})$ can be precisely described as:
$$out(N_i, C_{out_j}) = bias(C_{out_j} + \sum_{k=0}^{C_{in}-1}weight(C_{out_j},k)*input(N_i,k)$$
where $*$ is the valid 2D cross-correlation operator, $N$ is a batch size, $C$ denotes a number of channels, $H$ is a height of input planes in pixels, and $W$ is width in pixels.

### torch.nn.Linear

Applies a linear transformation to the incoming data: $y = xA^T + b$ 

### torch.nn.functional.max_pool2d

Applies a 2D **max pooling** over an input signal composed of several input planes.

>Max pooling reduces the dimensionality of an input by applying a max filter to usually non-overlapping subregions of the initial representation. Pooling layers are commonly inserted between successive convolution layers to reduce the amount of parameters and computation in the network, and to help control over fitting.

In the simplest case, the output value of the layer with input size $(N, C, H, W), output $(N, C, H_{out}, W_{out}) and `kernel_size` $(kH, kW)$ can be precisely described as:

$$out(N_i, C_j, H, W) = \max\limits_{m=0, \dots, kH-1} \max\limits_{n=0, \dots, kW-1} input(N_i, C_j, stride[0] * h + m, stride[1] * w + n)$$

### torch.nn.functional.relu

Applies the rectified linear unit function element-wise $ReLU(x) = max(0,x)$

<img src="https://pytorch.org/docs/stable/_images/ReLU.png" width=400/>


By defining the `forward` function, the `backward` function is automatically defined by `autograd`. The `forward` functio can unclude any Tensor operations.

The learnable parameters of a model are returned by `net.parameters()`.

In [3]:
params = list(net.parameters())
print(len(params))
print(params[0].size())   # conv1's .weight

10
torch.Size([6, 1, 5, 5])


Let try a random 32x32 input Note: Expected input size to this net(LeNet) is 32x32. To use this net on MNIST dataset, please resize the images from the dataset to 32x32.

In [4]:
input = torch.randn(1, 1, 32, 32)
out = net(input)
print(out)

tensor([[ 0.0136,  0.0495,  0.0561,  0.0133, -0.1004,  0.0282,  0.0582, -0.0402,
         -0.0497,  0.0237]], grad_fn=<ThAddmmBackward>)


Zero the gradient buffers of all parameters and backprops with random gradients:

In [5]:
net.zero_grad()
out.backward(torch.randn(1, 10))

>`torch.nn` only supports mini-batches. The entire `torch.nn` package only supports inputs that are a mini-batch of samples, and not a single sample. For example, `nn.Conv2d` will take in a 4D Tensor of nSamples x nChannels x Height x Width. If you have a single sample, just use `input.unsqueeze(0)` to add a fake batch dimension.

**Recap:**

* `torch.Tensor` - A *multi-dimensional array* with support for autograd operations like `backward()`.
* `nn.Module` - Neural network module. *Convenient way of encapsulating parameters*, with helpers for moving them to GPU, exporting, loading, etc.
* `nn.Parameter` - A kind of Tensor, that is *automatically registered as a parameter when assigned as an attribute to a `Module`.
* `autograd.Function` - Implements *forward and backward definitions of an autograd operation.* Every `Tensor` operation, creates at last a single `Function` node, that connects to functions that created a `Tensor` and *encodes its history.*

### Loss Function

A loss function takes the (output, target) pair of inputs, and computes a value that estimates how far away the output is from the target.

There are several different loss functions under the nn package. A simple loss is: `nn.MSELoss` which computes the mean-squared error between the input and the target.

For example:

In [6]:
output = net(input)
target = torch.randn(10)    # dummy target
target = target.view(1, -1) # make it the same shape as output
criterion = nn.MSELoss()

loss = criterion(output, target)
print(loss)

tensor(1.2344, grad_fn=<MseLossBackward>)


### nn.MSELoss

Creates a criterion that measures the mean squared error between $n$ elements in the input $x$ and target $y$.

The loss can be described as:
$$l(x,y) = L = \{l_1, \dots, l_N\}^\top, l_n = (x_n - y_n)^2$$
Where N is the batch size. If reduce is `True`, then:
$$l(x,y) = \begin{cases} mean(L), & \mbox{if } size\_average\mbox{ = True,} \\sum(L), & \mbox{if } size\_average\mbox{ = False} \end{cases}$$
The sum operation still operates over all the elements, and divides by $n$. 

In [7]:
loss.grad_fn